In [1]:
import numpy as np
import gurobipy as gp

from graph import Graph
from bssrp_mip import BSSRPMIP
from scipy.spatial.distance import pdist, squareform

### Generate Graph

In [2]:
seed = 1234

In [3]:
use_penalties = True

In [4]:
num_nodes = 20
num_vehicles = 3
time_limit = 60

penalty_cost_demand = 1
penalty_cost_time = 1

speed = 30 

In [5]:
g = Graph(
        num_nodes = num_nodes, 
        k_nn = 2, 
        num_vehicles = num_vehicles,
        penalty_cost_demand = penalty_cost_demand,
        penalty_cost_time = penalty_cost_time, 
        speed = speed,
        time_limit = time_limit)

In [6]:
g.seed(seed)

In [7]:
g.bss_graph_gen()

### Initialize MIP

In [8]:
mip = BSSRPMIP(g, use_penalties=use_penalties)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-01-15
Set parameter TimeLimit to value 100000


In [9]:
mip.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 2594 rows, 3480 columns and 12714 nonzeros
Model fingerprint: 0xfb799156
Variable types: 3 continuous, 3477 integer (1197 binary)
Coefficient statistics:
  Matrix range     [4e-01, 2e+01]
  Objective range  [4e-01, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 114 rows and 60 columns
Presolve time: 0.02s
Presolved: 2480 rows, 3420 columns, 12483 nonzeros
Variable types: 0 continuous, 3420 integer (1197 binary)

Root relaxation: objective 7.056093e+01, 4589 iterations, 0.17 seconds (0.26 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   70.56093    0  143          -   70.56093      -     -    0s
     0     0   70.56461    0  202          -   70.56461      -    

In [10]:
mip.get_routes()

{0: [0, 3, 18, 19, 9, 4, 12, 15, 17, 6, 1, 13, 0],
 1: [0, 14, 10, 7, 5, 2, 16, 8, 11, 0],
 2: []}

In [10]:
mip.print_routes()

Vehicle 0:
    Route: [0, 3, 18, 19, 9, 4, 12, 15, 17, 6, 1, 13, 0]
    Cost: 48.503249408498455 

Vehicle 1:
    Route: []
    Cost: 0 

Vehicle 2:
    Route: [0, 14, 10, 7, 5, 2, 16, 8, 11, 0]
    Cost: 41.82269160401945 

